In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- 1. Import the dataset ---
DATASET_URL = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
df = pd.read_csv(DATASET_URL)

# --- 3. Handle missing values & 6. Correct data types (for TotalCharges) ---

df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median()).astype('float64')



# Correct other basic data types
df['tenure'] = df['tenure'].astype('int64')
df['MonthlyCharges'] = df['MonthlyCharges'].astype('float64')
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object') # Treat 0/1 as a category for now
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0}).astype('int64') # Target variable conversion

# --- 4. Remove duplicate records ---
df.drop_duplicates(inplace=True)
df.drop('customerID', axis=1, inplace=True) 

# --- 5. Check for inconsistent data and standardize it ---
for col in ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']:
    df[col] = df[col].replace('No internet service', 'No')
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')

# --- 7. Identify and handle outliers (Capping using IQR) ---
numerical_cols_for_outliers = ['tenure', 'MonthlyCharges', 'TotalCharges']

for col in numerical_cols_for_outliers:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR


    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])


# --- 8. Perform feature engineering ---
# Create a 'HasInternet' feature 
df['HasInternet'] = df['InternetService'].apply(lambda x: 0 if x == 'No' else 1)
# Create 'Total_Monthly_Ratio'
df['Total_Monthly_Ratio'] = df['TotalCharges'] / df['MonthlyCharges']
# Replace infinite values that might result from division by very small/zero MonthlyCharges
df['Total_Monthly_Ratio'].replace([np.inf, -np.inf], 0)


# --- 9. Normalize or scale the data 
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'Total_Monthly_Ratio', 'HasInternet']

# One-hot encode categorical features
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Prepare numerical data for scaling (using only columns that need standard scaling)
cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges', 'Total_Monthly_Ratio']
scaler = StandardScaler()
df_encoded[cols_to_scale] = scaler.fit_transform(df_encoded[cols_to_scale])

# --- 10. Split the dataset into training and testing sets ---
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- 11. Export the cleaned dataset for future analysis or modeling ---
df_encoded.to_csv("Telecom_Customer_Churn_FINAL_CLEANED.csv", index=False)
print(df)

      gender SeniorCitizen Partner Dependents  tenure PhoneService  \
0     Female             0     Yes         No     1.0           No   
1       Male             0      No         No    34.0          Yes   
2       Male             0      No         No     2.0          Yes   
3       Male             0      No         No    45.0           No   
4     Female             0      No         No     2.0          Yes   
...      ...           ...     ...        ...     ...          ...   
7038    Male             0     Yes        Yes    24.0          Yes   
7039  Female             0     Yes        Yes    72.0          Yes   
7040  Female             0     Yes        Yes    11.0           No   
7041    Male             1     Yes         No     4.0          Yes   
7042    Male             0      No         No    66.0          Yes   

     MultipleLines InternetService OnlineSecurity OnlineBackup  ...  \
0               No             DSL             No          Yes  ...   
1               N